In [1]:
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from urllib.parse import urlparse


In [2]:
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

# Inicialização do WebDriver
driver = webdriver.Chrome(options=options)
driver.maximize_window()

urls = []
base_url = "https://www.chavesnamao.com.br/"

# Define o número de páginas a serem percorridas
num_paginas = 100

for i in range(1, num_paginas + 1):
    url = f"{base_url}/casas-a-venda/brasil/?pg={i}"
    driver.get(url)

    # Faz scroll para carregar mais anúncios, se necessário
    last_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(0, 30):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Coletar os links dos anúncios com a nova classe
    soup = BeautifulSoup(driver.page_source, "html.parser")
    anuncios = soup.find_all("a", {"class": "link-module__x-ue-q__rawLink column"})
    for anuncio in anuncios:
        link = anuncio["href"]
        if link.startswith("/"):
            link = base_url + link
        urls.append(link)

# Finalizando o driver
driver.quit()

print(f"Total de links coletados: {len(urls)}")

Total de links coletados: 2884


In [3]:
urls = list(set(urls))
pd.DataFrame(urls).to_csv("urls_chave_na_mao.csv", index=False, header=False)
print(len(urls))

1495


In [4]:
df = pd.DataFrame()
# Funções auxiliares
def deve_processar_url(url):
    """Verifica se a URL deve ser processada (não começa com www)"""
    parsed = urlparse(url)
    return not parsed.netloc.startswith('www.')

def extrair_acabamentos(texto):
    padrao = r"porcelanato|ceramica|cerâmica|cerâmico|cerâmicos|gesso|antiderrapante|planejado|tábua"
    if isinstance(texto, str):
        encontrados = re.findall(padrao, texto.lower())
        return ", ".join(set(encontrados)) if encontrados else None
    return None

def extrair_tipo_imovel(texto):
    padrao = r"casa|apartamento|sobrado|fazenda"
    if isinstance(texto, str):
        encontrados = re.findall(padrao, texto.lower())
        return ", ".join(set(encontrados)) if encontrados else None
    return None

scraper = cloudscraper.create_scraper()
# Remove duplicatas
urls = list(set(urls))  

for url in urls:
    time.sleep(1)
    #Pula as urls que iniciam com www
    try:
        if not deve_processar_url(url):
            print(f"Pulando URL não permitida: {url}")
            continue
        
        response = scraper.get(url, headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"})

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            dados_imovel = {"Link": url}

            # Busca Localização
            location = soup.find("div", {"id": "location"})
            if location:
                endereco_texto = location.get_text(" ", strip=True)
                cep = re.findall(r"\d{5}-?\d{3}", endereco_texto)
                cep = cep[-1] if cep else None
                endereco_limpo = endereco_texto.replace("Localização", "")\
                                            .replace("Exibir no mapa", "")\
                                            .replace(cep, "")\
                                            .strip()
                endereco_limpo = re.sub(r'[,\s-]+$', '', endereco_limpo).strip()
                dados_imovel.update({
                    "Endereço": endereco_limpo,
                    "CEP": cep
                })

            # Busca Valor
            div_valor_mae = soup.find("div", {"id": "price-box-container"}) 
            if div_valor_mae:
                div_valor = div_valor_mae.find("div", {"class": "olx-d-flex olx-fd-column"}) if div_valor_mae else None
                if div_valor:
                    valor_texto = div_valor.get_text(strip=True)
                    valor_limpo = re.sub(r'[^\d,]', '', valor_texto).replace(',', '.').strip()
                    dados_imovel["Valor Imóvel"] = valor_limpo if valor_limpo else None


            # Busca características do imóvel
            div_principal = soup.find("div", {"class": "ad__sc-wuor06-0 hfcCRQ"})
            if div_principal:
                divs_caracteristicas = div_principal.find_all("div", recursive=False)
                
                for div in divs_caracteristicas:
                    texto = div.get_text(strip=True)
                    if "Área construída" in texto:
                        dados_imovel["Área construída"] = re.search(r"(\d+)", texto).group(1) if re.search(r"(\d+)", texto) else None
                    elif "Quartos" in texto:
                        dados_imovel["Quartos"] = texto.replace("Quartos", "").strip()
                    elif "Banheiros" in texto:
                        dados_imovel["Banheiros"] = texto.replace("Banheiros", "").strip()
                    elif "Vagas na garagem" in texto:
                        dados_imovel["Vagas na garagem"] = texto.replace("Vagas na garagem", "").strip()

            # Descrição do imóvel
            descricao = soup.find("div", {"id": "description-title"})
            descricao_texto = descricao.get_text(" ", strip=True) if descricao else None
            dados_imovel["Descrição"] = descricao_texto

            # Busca detalhes adicionais
            if descricao:
                desc_texto = descricao.get_text(" ", strip=True)
                dados_imovel["Acabamentos"] = extrair_acabamentos(desc_texto)
                dados_imovel["Tipo Imóvel"] = extrair_tipo_imovel(desc_texto)

            df = pd.concat([df, pd.DataFrame([dados_imovel])], ignore_index=True)
            print(dados_imovel)
    except Exception as e:
        print(f"Erro ao processar a URL {url}: {e}")
        continue


Pulando URL não permitida: https://www.chavesnamao.com.br//imovel/casa-a-venda-6-quartos-com-garagem-pr-cascavel-claudete-412m2-RS630000/id-11665470/
Pulando URL não permitida: https://www.chavesnamao.com.br//imovel/casa-a-venda-3-quartos-com-garagem-sp-sao-bernardo-do-campo-rudge-ramos-82m2-RS530000/id-25759290/
Pulando URL não permitida: https://www.chavesnamao.com.br//imovel/casa-a-venda-3-quartos-com-garagem-sc-balneario-camboriu-nova-esperanca-140m2-RS600000/id-11670428/
Pulando URL não permitida: https://www.chavesnamao.com.br//imovel/casa-a-venda-5-quartos-com-garagem-rj-rio-de-janeiro-vila-valqueire-760m2-RS1500000/id-9050112/
Pulando URL não permitida: https://www.chavesnamao.com.br//imovel/casa-a-venda-3-quartos-com-garagem-sp-praia-grande-solemar-270m2-RS1190000/id-27078642/
Pulando URL não permitida: https://www.chavesnamao.com.br//imovel/casa-a-venda-3-quartos-com-garagem-sp-rio-das-pedras-jardim-dona-antonieta-RS380000/id-29523722/
Pulando URL não permitida: https://www.c

In [5]:
df.to_excel("databases/imoveis.xlsx", index=False)

OSError: Cannot save file into a non-existent directory: 'databases'